In [1]:
from transformers import pipeline
from openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


/Users/caochunqin/Desktop/githomework/milestone2_pt2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
def img2text(url):
    img_to_text_pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
    text = img_to_text_pipe(url)[0]["generated_text"]
    # print(text)
    return text

In [4]:
scenario = img2text("library.jpg")
print(scenario)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/caochunqin/Desktop/githomework/milestone2_pt2/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


bookshelves with many different colored books on them in a library


In [5]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=200,
    timeout=None,
    max_retries=2
)

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert short story teller. Using a simple narrative you generate story in less than 100 words based on the given scenario.",
        ),
        ("human", "{scenario_lang}"),
    ]
)

In [7]:
chain = prompt | llm | StrOutputParser()

chain.invoke({
    "scenario_lang" : scenario
})

'In the quiet library, sunlight streamed through tall windows, casting rainbows on the bookshelves. Each book, a different color, whispered secrets of distant lands and forgotten times. A young girl, eyes wide with wonder, reached for a vibrant blue spine. As she opened it, the room seemed to hold its breath. The pages rustled softly, revealing tales of ocean voyages and mythical creatures. She smiled, feeling the magic of the library envelop her. In that moment, surrounded by a kaleidoscope of stories, she knew she had found her own adventure waiting to be written.'

In [9]:
def textGeneration_langChain(msg,type):
    """
    msg is the scenario for the story from the pic (hugging face model output);
    type is the genre of the story- Horror, Fantasy, Adventure, Comedy, Mystery, Romance
    """
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.2,
        max_tokens=200,
        timeout=None,
        max_retries=2
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert short {story_type} story teller. Using a simple narrative you generate {story_type} story in less than 100 words based on the given scenario.",
            ),
            (
                "human", 
                "{scenario_lang}"
            ),
        ]
    )

    chain = prompt | llm | StrOutputParser()

    out_message = chain.invoke({
        "story_type" : type,
        "scenario_lang" : msg,
    })

    return out_message

    

In [10]:
story = textGeneration_langChain(scenario,"Horror")
print(story)

In the dimly lit library, the bookshelves stood like sentinels, their spines a kaleidoscope of colors. As I reached for a crimson tome, it slipped from my grasp, landing with a hollow thud. The room seemed to exhale, and the books began to whisper, their voices a cacophony of forgotten secrets. I backed away, but the shelves loomed closer, the colors swirling into a vortex of madness. The whispers grew louder, a chant of my name, pulling me in. The last thing I saw was the crimson book opening, its pages blank, waiting to write my story.


In [11]:
def runModels_langchain(url, type):
    scenario = img2text(url)
    story = textGeneration_langChain(scenario,type)
    return([scenario,story])

In [12]:
(caption, story) = runModels_langchain("library.jpg","Comedy")
print(story)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/caochunqin/Desktop/githomework/milestone2_pt2/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In the quiet library, a mischievous librarian named Mr. Page had a secret hobby: color-coding chaos. One day, he rearranged the books so that each shelf formed a rainbow. Patrons were baffled. "Excuse me," a woman asked, "where's 'War and Peace'?" Mr. Page grinned, "Under 'W' for 'Wow, that's colorful!'" 

A man searching for a mystery novel found himself in the romance section, confused by the pink spines. "Is this a clue?" he wondered aloud. Mr. Page winked, "Only if you love a good mystery!" 

And so, the library became a place where finding a book was the real adventure.
